In [1]:
class Model:
    '''
    The default model is a Logistic Regression.
    '''
    def __init__(self, train, cols_bin, y):
        self.C = 1
        self.train = train
        self.cols_bin = cols_bin
        self.y = y
        self.model = linear_model.LogisticRegression(C = self.C)
        self.mfit = self.model.fit(self.train[self.cols_bin], self.train[self.y])
        
    def predict(self, df):
        self.df = df
        self.prediction = self.mfit.predict(df[self.cols_bin])[0]
        
    def predict_proba(self, df):
        self.df = df
        self.prob = self.mfit.predict_proba(df[self.cols_bin])

In [2]:
class SVM_Model(Model):
    '''Support Vector Machine'''
    def __init__(self, train, cols_bin, y):
        self.C = 1
        self.train = train
        self.cols_bin = cols_bin
        self.y = y
        self.model = SVC(C=self.C)
        self.mfit = self.model.fit(self.train[self.cols_bin], self.train[self.y])

In [3]:
class LinearSVM_Model(Model):
    '''Linear Support Vector Machine'''
    def __init__(self, train, cols_bin, y):
        self.C = 1
        self.train = train
        self.cols_bin = cols_bin
        self.y = y
        self.model = LinearSVC(C=self.C)
        self.mfit = self.model.fit(self.train[self.cols_bin], self.train[self.y])

In [4]:
class Gauss_Model(Model):
    '''Gaussian Naive Bayes'''
    def __init__(self, train, cols_bin, y):
        self.C = 1
        self.train = train
        self.cols_bin = cols_bin
        self.y = y
        self.model = GaussianNB()
        self.mfit = self.model.fit(self.train[self.cols_bin], self.train[self.y])

In [5]:
class MLP_Model(Model):
    '''Multi-Layer Perceptron Classifier'''
    def __init__(self, train, cols_bin, y):
        self.C = 1
        self.train = train
        self.cols_bin = cols_bin
        self.y = y
        self.model = MLPClassifier(alpha=1e-5, max_iter=500,
                                  hidden_layer_sizes=3 * [500],
                                  random_state=1)
        self.mfit = self.model.fit(self.train[self.cols_bin], self.train[self.y])

In [6]:
class RiskManager:
    '''Computes accuracy scores and attempts to calculate Kelly Criterion.'''
    def __init__(self, train, cols_bin, mfit, y):
        self.train = train
        self.cols_bin = cols_bin
        self.mfit = mfit
        self.y = y
        self.train['strat_pos'] = self.mfit.predict(self.train[self.cols_bin])
        self.train['strat_ret'] = self.train['strat_pos'] * self.train['Returns']
        NN = len(self.train[(self.train['strat_pos'] == -1) & (self.train['direction'] == -1)])
        NP = len(self.train[(self.train['strat_pos'] == -1) & (self.train['direction'] == 1)])
        PN = len(self.train[(self.train['strat_pos'] == 1) & (self.train['direction'] == -1)])
        PP = len(self.train[(self.train['strat_pos'] == 1) & (self.train['direction'] == 1)])
        self.score = accuracy_score(self.train[self.y], self.train['strat_pos'])
        self.precision = (PP) / (PP + PN)
        self.recall = (PP) / (PP + NP)
        self.f1 = 2 * ((self.precision * self.recall) / (self.precision + self.recall))
        W = (NN + PP) / (NN + NP + PN + PP)
        gains = list(self.train[((self.train['strat_pos'] == 1) & (self.train['direction'] == 1)) | 
                        ((self.train['strat_pos'] == -1) & (self.train['direction'] == -1))].strat_ret)
        losses = list(self.train[((self.train['strat_pos'] == 1) & (self.train['direction'] == -1)) | 
                        ((self.train['strat_pos'] == -1) & (self.train['direction'] == 1))].strat_ret)

        R = (sum(gains) / len(gains)) / (sum(losses) / len(losses))

        self.K = W - ((1 - W)/R)
        
    def pos_size(self, balance):
        pos_size = (self.K * balance)
        return pos_size